# Importing Stuff

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer, OneHotEncoder

from sentence_transformers import SentenceTransformer
from langdetect import detect, LangDetectException

import re
import nltk
from nltk.corpus import stopwords

2024-10-23 13:00:50.067799: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Reading the cleaned dataset

In [2]:
df = pd.read_csv("data/dataset_cleaned.csv")
df.head()

,Title,Author,Price,List Price,Discount Amount,Limited Stock,Discount,Genre,Number of Pages,Weight,ISBN,Language,Related Genres,Subgenres,Synopsis,URL
0,The Inner Game of Music,"Barry Green, W. Timothy Gallwey",1040.0,1040.0,0.0,True,0.0,Arts And Photography,248.0,200,9781447291725,English,"['Arts And Photography', 'Self Improvement And...","['Music', 'Self Help', 'Psychology']",The bestselling guide to improving musical per...,https://booksmandala.com/books/the-inner-game-...
1,The Nepalis a pictorial celebration,"Deepak Thapa, Narendra. Pradhan",1500.0,1500.0,0.0,True,0.0,Arts And Photography,116.0,515,9789993347378,English,"['Arts And Photography', 'Nepali']","['Picture Books', 'Books on Nepal']",Visitors to Nepal are very often struck by the...,https://booksmandala.com/books/the-nepalis-a-p...
2,Lust for Life,Irving Stone,1118.0,1118.0,0.0,True,0.0,History Biography And Social Science,479.0,370,9781787461390,English,"['History Biography And Social Science', 'Arts...","['Biography', 'Memoir', 'Art']","Since its initial publication in 1934, Irving ...",https://booksmandala.com/books/lust-for-life-3...
3,The Architecture Book,Dk Publishing,2238.0,2238.0,0.0,True,0.0,Arts And Photography,335.0,1200,9780241415030,English,"['Arts And Photography', 'Learning And Referen...","['Architecture', 'Science']",Discover the key architectural concepts behind...,https://booksmandala.com/books/the-architectur...
4,The World According to Colour,James Fox,1438.0,1438.0,0.0,True,0.0,History Biography And Social Science,295.0,290,9780141976655,English,"['History Biography And Social Science', 'Arts...","['History', 'Design', 'Art', 'Science']","A beguiling cultural history of colour, by one...",https://booksmandala.com/books/the-world-accor...


# Preprocess

## Get rid of missing rows

In [3]:
na = df.isna().sum()
na[na > 0]

Author         3
Subgenres    136
dtype: int64

In [4]:
df.dropna(subset=["Author"], inplace=True)

## Remove "Unassigned" Authors

In [5]:
df["Author"].value_counts()

Author
Unassigned                      27
Jeff Kinney                     24
Hergé                           23
Thich Nhat Hanh                 16
Kentaro Miura                   16
                                ..
Luo Di Cheng Qiu                 1
Jim Starlin                      1
Negi Haruba                      1
Syougo Kinugasa, Yuyu Ichino     1
Peter Matthiessen                1
Name: count, Length: 1347, dtype: int64

In [6]:
df = df[df["Author"] != "Unassigned"]

## Remove Redundant Synopses

In [7]:
df["Language"].value_counts()

Language
English     1774
Nepali       116
French        13
Hindi         13
German         3
Japanese       1
Bengali        1
Name: count, dtype: int64

In [8]:
df[df["Language"] != "English"]["Synopsis"]

109     Timeless lessons on wealth, greed, and happine...
258     " Je me représente très bien l'époque où les r...
259     फ़ैज़ अहमद फै़ज़ उर्दू के बहुत ही जाने-माने कव...
260     ऋषि अष्टावक्र और राजा जनक के संवाद का संकलन है...
262     本書ではネパール・ヒマラヤの中でも、交通の便、現地事情、内外トレッキング・エージェントの実績...
                              ...                        
1191    त्यो मेरो अनुभूतिभन्दा बाहिरको मूल्य हो आदिमात...
1192    Sumnima a famous Nepali novel by B P Koirala, ...
1321         Hindi verse translation of the Bhagavadgītā.
1388    धर्मलाई बुझ्ने मानिस दुर्लभ हुन्छ, बुझेर अरूला...
1639    This collection of new translations brings tog...
Name: Synopsis, Length: 147, dtype: object

In [9]:
df.duplicated(subset=["ISBN", "Synopsis"]).sum()

0

In [10]:
df[df["Language"] != "English"]["Synopsis"]

109     Timeless lessons on wealth, greed, and happine...
258     " Je me représente très bien l'époque où les r...
259     फ़ैज़ अहमद फै़ज़ उर्दू के बहुत ही जाने-माने कव...
260     ऋषि अष्टावक्र और राजा जनक के संवाद का संकलन है...
262     本書ではネパール・ヒマラヤの中でも、交通の便、現地事情、内外トレッキング・エージェントの実績...
                              ...                        
1191    त्यो मेरो अनुभूतिभन्दा बाहिरको मूल्य हो आदिमात...
1192    Sumnima a famous Nepali novel by B P Koirala, ...
1321         Hindi verse translation of the Bhagavadgītā.
1388    धर्मलाई बुझ्ने मानिस दुर्लभ हुन्छ, बुझेर अरूला...
1639    This collection of new translations brings tog...
Name: Synopsis, Length: 147, dtype: object

In [11]:
detect(df['Synopsis'][291])

'nl'

In [12]:
df.iloc[291]

Title                                    Flickan som lekte med elden
Author                                                 Stieg Larsson
Price                                                          400.0
List Price                                                     400.0
Discount Amount                                                  0.0
Limited Stock                                                   True
Discount                                                         0.0
Genre                                              Foreign Languages
Number of Pages                                                260.0
Weight                                                           250
ISBN                                                   9789170014833
Language                                                     English
Related Genres                                 ['Foreign Languages']
Subgenres                                                 ['German']
Synopsis           On the eve of p

In [13]:
def detect_language(row):
    try:
        return detect(row['Synopsis']) 
    except LangDetectException:
        return row['Language']

df['Language'] = df.apply(detect_language, axis=1)

In [14]:
df['Language'].unique()

array(['en', 'fr', 'hi', 'ja', 'nl', 'de', 'ne', 'English', 'cy', 'lv',
       'no'], dtype=object)

In [15]:
df[df['Language'] == 'English']

,Title,Author,Price,List Price,Discount Amount,Limited Stock,Discount,Genre,Number of Pages,Weight,ISBN,Language,Related Genres,Subgenres,Synopsis,URL
413,George's Marvellous Medicine,Roald Dahl,560.0,560.0,0.0,False,0.0,Kids And Teens,136.0,200,9780241558485,English,"['Kids And Teens', 'Fiction And Literature']","['Childrens', 'Fantasy', 'Classics', 'Young Ad...",560,https://booksmandala.com/books/georges-marvell...


In [16]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

def get_synopsis(isbn: str) -> str | None:
    # GET YOUR OWN API KEY!!!
    response = requests.get(f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}&key={os.getenv('API_KEY')}")
    data = response.json()

    if "items" not in data or len(data["items"]) == 0:
        return None
        
    new_response = requests.get(data["items"][0].get("selfLink", {}))
    new_data = new_response.json()
    new_data = new_data.get("volumeInfo", {})
    return new_data.get("description", None)

df["Synopsis"] = df.apply(lambda row: row["Synopsis"] if row["Language"] != "English" else get_synopsis(row["ISBN"]), axis=1)

In [17]:
df["Language"] = df.apply(lambda row: row["Language"] if row["Language"] != "English" else "en", axis=1)

In [18]:
df["Synopsis"] = df["Synopsis"].str.replace(r"<.*?>", '', regex=True)

## Safe parse genres and subgenres

In [19]:
import ast

def safe_eval_list(x: str | list[str]) -> list:
    if pd.isna(x):
        return []

    if isinstance(x, list):
        return x

    try:
        evaluated = ast.literal_eval(x)
        if isinstance(evaluated, list):
            return evaluated
    except:
        return []

In [20]:
df["Related Genres"] = df["Related Genres"].apply(safe_eval_list)

In [21]:
df["Subgenres"] = df["Subgenres"].apply(safe_eval_list)

In [22]:
df[["Related Genres", "Subgenres"]]

,Related Genres,Subgenres
0,"[Arts And Photography, Self Improvement And Re...","[Music, Self Help, Psychology]"
1,"[Arts And Photography, Nepali]","[Picture Books, Books on Nepal]"
2,"[History Biography And Social Science, Arts An...","[Biography, Memoir, Art]"
3,"[Arts And Photography, Learning And Reference]","[Architecture, Science]"
4,"[History Biography And Social Science, Arts An...","[History, Design, Art, Science]"
...,...,...
1946,[History Biography And Social Science],[Memoir]
1947,"[Travel, Nepali]","[Climbing and Mountaineering, Books on Nepal]"
1948,[Fiction And Literature],"[Classics, Contemporary]"
1949,"[History Biography And Social Science, Busines...","[Memoir, Biography, Business]"


In [23]:
df.dropna(subset=["Synopsis"], inplace=True)

## **Related Genres** and **Subgenres**

In [24]:
mlb_genres = MultiLabelBinarizer()

genres_encoded = pd.DataFrame(mlb_genres.fit_transform(df["Related Genres"]), 
                              columns=mlb_genres.classes_, 
                              index=df.index)

In [25]:
genres_encoded

,Arts And Photography,Business And Investing,Fiction And Literature,Foreign Languages,History Biography And Social Science,Kids And Teens,Learning And Reference,Lifestyle And Wellness,Manga And Graphic Novels,Miscellaneous,Nature,Nepali,Rare Coffee Table Books,Religion,Self Improvement And Relationships,Spirituality And Philosophy,Technology,Travel
0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1947,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1948,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1949,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
mlb_subgenres = MultiLabelBinarizer()

subgenres_encoded = pd.DataFrame(mlb_subgenres.fit_transform(df["Subgenres"]), 
                              columns=mlb_subgenres.classes_, 
                              index=df.index)

In [27]:
print(len(subgenres_encoded.columns))
subgenres_encoded.columns

139


Index(['Action and Adventure', 'Adult Fiction', 'Ages 3 to 5', 'Ages 6 to 8',
       'Ages 9 to 12', 'Animals and Pets', 'Anthology', 'Anthropology',
       'Architecture', 'Art',
       ...
       'Stress Management', 'Tarot', 'Teens and Young Adult',
       'Thriller and Suspense', 'Time Management', 'Travel Guide Books',
       'Trees and Plants', 'True Crime', 'Womens Fiction', 'Young Adult'],
      dtype='object', length=139)

One hot encoding all related genres and subgenres will result in $18 + 139 = 157$ additional columns, increasing the dimensionality.

## Encoding

In [28]:
df['genres_str'] = df["Related Genres"].apply(lambda x: ','.join(x))
df['subgenres_str'] = df['Subgenres'].apply(lambda x: ','.join(x))

In [29]:
df = df[["Title", "Author", "Genre", "Language", "Synopsis", "genres_str", "subgenres_str", "ISBN", "URL"]]

In [30]:
df["combined_genres"] = df["genres_str"] + "," + df["subgenres_str"]

In [31]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [32]:
df.reset_index(inplace=True)
df["synopsis_embeddings"] = list(model.encode(df["Synopsis"], show_progress_bar=True))

Batches:   0%|          | 0/61 [00:00<?, ?it/s]

### Treating Genres and Subgenres as Combined Features

In [33]:
tfidf = TfidfVectorizer(stop_words='english')
genre_vectors = tfidf.fit_transform(df["combined_genres"])

### Combine and Add Weights

In [34]:


weights = {
    "genre_weight": 3.0,
    "synopsis_weight": 2.0
}

synopsis_embeddings_matrix = np.vstack(df["synopsis_embeddings"].values)

combined_features = hstack([
    weights["genre_weight"] * genre_vectors,
    weights["synopsis_weight"] * synopsis_embeddings_matrix
])

# Calculate Similarity

## Cosine Similarity

In [35]:
similarity_matrix = cosine_similarity(combined_features)
similarity_df = pd.DataFrame(similarity_matrix, index=df['Title'], columns=df['Title'])

In [36]:
def get_recommendations(similarity_df: pd.DataFrame,
                        df: pd.DataFrame,
                        title: str, 
                        n: int = 5, 
                        columns: list[str] = ["Title", "Author"]
                       ) -> pd.DataFrame:
        idx = similarity_df.index.get_loc(title)
        sim_scores = list(enumerate(similarity_df.iloc[idx]))
    
        # sort the books based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
        sim_scores = sim_scores[1:n+1]  # exclude the first one as it's the book itself
    
        # get the book indices
        book_indices = [i[0] for i in sim_scores]
    
        # top n most similar books
        return df[columns].iloc[book_indices]

In [37]:
get_recommendations(similarity_df, df, "Peter Pan", n=10, columns=["Title", "Author", "combined_genres", "Language", "URL"])

,Title,Author,combined_genres,Language,URL
346,Pater Pan,Fun Studio,"Kids And Teens,Baby to 2,Ages 3 to 5",en,https://booksmandala.com/books/pater-pan-49862
438,The Jungle Book,Miriam Bos,"Arts And Photography,Kids And Teens,Picture Bo...",en,https://booksmandala.com/books/the-jungle-book...
50,The 7 Habits of Happy Kids,Sean Covey,"Arts And Photography,Kids And Teens,Picture Bo...",en,https://booksmandala.com/books/the-7-habits-of...
8,Colouring book : Copy Colour Fruits and Vegeta...,Om Books International,"Kids And Teens,Arts And Photography,Ages 3 to ...",en,https://booksmandala.com/books/colouring-book-...
37,Colouring book : Copy Colouring Book Noisy Din...,Om Books International,"Kids And Teens,Arts And Photography,Ages 3 to ...",en,https://booksmandala.com/books/colouring-book-...
446,"Gruffalo, Where Are You?: a Felt Flaps Book",Julia Donaldon,"Arts And Photography,Kids And Teens,Picture Bo...",en,https://booksmandala.com/books/gruffalo-where-...
688,Destination Moon,Hergé,"Kids And Teens,Manga And Graphic Novels,Ages 9...",en,https://booksmandala.com/books/destination-moo...
665,Explorers on the Moon,"Hergé, Leslie Lonsdale-Cooper, Michael R. Turner","Kids And Teens,Manga And Graphic Novels,Ages 9...",en,https://booksmandala.com/books/explorers-on-th...
357,Brain Activity Book for Kids - 200+ Activities...,Maple Press,"Kids And Teens,Ages 3 to 5,Ages 6 to 8,Childre...",en,https://booksmandala.com/books/brain-activity-...
657,The Calculus Affair,Hergé,"Kids And Teens,Manga And Graphic Novels,Ages 9...",en,https://booksmandala.com/books/the-calculus-af...


In [38]:
get_recommendations(similarity_df, df, "Diary of a Wimpy Kid", n=10, columns=["Title", "Author", "combined_genres", "Language", "URL"])

,Title,Author,combined_genres,Language,URL
341,Diary Of A Wimpy Kid ; The Ugly Truth,Jeff Kinney,"Kids And Teens,Ages 9 to 12",en,https://booksmandala.com/books/diary-of-a-wimp...
362,Diary of a Wimpy Kid: Wrecking Ball,Jeff Kinney,"Kids And Teens,Ages 9 to 12",en,https://booksmandala.com/books/diary-of-a-wimp...
355,Cabin Fever,Jeff Kinney,"Kids And Teens,Ages 9 to 12",en,https://booksmandala.com/books/cabin-fever-37914
787,Diary Of A Wimpy Kid: No Brainer,Jeff Kinney,"Kids And Teens,Fiction And Literature,Manga An...",en,https://booksmandala.com/books/diary-of-a-wimp...
741,Diary of an Awesome Friendly Kid: Rowley Jeffe...,Jeff Kinney,"Fiction And Literature,Kids And Teens,Manga An...",en,https://booksmandala.com/books/diary-of-an-awe...
359,Rodrick Rules,Jeff Kinney,"Kids And Teens,Ages 6 to 8,Ages 9 to 12",en,https://booksmandala.com/books/rodrick-rules-3...
371,Diary of a Wimpy Kid 10. Old School,Jeff Kinney,"Fiction And Literature,Kids And Teens,Young Ad...",en,https://booksmandala.com/books/diary-of-a-wimp...
434,Diary Of A Wimpy Kid ; Hard Luck,Jeff Kinney,"Kids And Teens,Ages 9 to 12",en,https://booksmandala.com/books/diary-of-a-wimp...
377,The Wimpy Kid Movie Diary,Jeff Kinney,"Kids And Teens,Ages 9 to 12",en,https://booksmandala.com/books/the-wimpy-kid-m...
427,Diary of a Wimpy Kid: Diper Overlode (Book 17),Jeff Kinney,"Kids And Teens,Ages 9 to 12",en,https://booksmandala.com/books/diary-of-a-wimp...


In [39]:
get_recommendations(similarity_df, df, "Satori in Paris", n=5, columns=["Title", "Author", "combined_genres", "Language", "URL"])

,Title,Author,combined_genres,Language,URL
1878,Lonesome Traveler,Jack Kerouac,"Fiction And Literature,Classics",en,https://booksmandala.com/books/lonesome-travel...
1610,Metamorphosis and Other Stories,Franz Kafka,"Fiction And Literature,Spirituality And Philos...",en,https://booksmandala.com/books/metamorphosis-a...
1898,On the Road,"Jack Kerouac, Ann Charters","Fiction And Literature,Classics",en,https://booksmandala.com/books/on-the-road-49631
1561,Metamorphosis and Other Stories,Franz Kafka,"Fiction And Literature,Spirituality And Philos...",en,https://booksmandala.com/books/metamorphosis-a...
226,Norwegian Wood,Haruki Murakami,"Fiction And Literature,Contemporary,Romance",en,https://booksmandala.com/books/norwegian-wood-...


## K-Nearest Neighbors

In [41]:
# scaling the features

# scaling the synopsis (dense)
scaler = StandardScaler()
synopsis_embeddings_scaled = scaler.fit_transform(synopsis_embeddings_matrix)

combined_features_scaled = np.hstack([
    synopsis_embeddings_scaled,
    genre_vectors.toarray()
])

In [45]:


k = 10 # number of recommendations (neighbors)

# KNN using cosine distance
knn = NearestNeighbors(n_neighbors=k, metric='cosine', algorithm='brute')
knn.fit(combined_features_scaled)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [52]:
def recommend_knn(book_title, 
                  df, 
                  knn_model, 
                  combined_features_scaled, 
                  n_recommendations=5,
                  columns=['Title', 'Author', 'combined_genres', 'Distance']):
    try:
        book_idx = df[df['Title'] == book_title].index[0]
    except IndexError:
        return f'Book "{book_title}" not found.'

    # Get the feature vector for the book
    book_vector = combined_features_scaled[book_idx].reshape(1, -1)
    
    # Find k nearest neighbors (including the book itself)
    distances, indices = knn_model.kneighbors(book_vector, n_neighbors=n_recommendations+1)
    
    # Get indices of the recommended books (skipping the first as it's the book itself)
    recommended_indices = indices[0][1:]
    recommended_distances = distances[0][1:]
    
    # Create a DataFrame with the recommended books and distances
    recommendations_df = df.iloc[recommended_indices].copy()  # Copy to avoid SettingWithCopyWarning
    recommendations_df['Distance'] = recommended_distances  # Add distances to the DataFrame
    
    return recommendations_df[columns]

In [53]:
recommend_knn("Satori in Paris", df, knn, combined_features_scaled)

,Title,Author,combined_genres,Distance
1878,Lonesome Traveler,Jack Kerouac,"Fiction And Literature,Classics",0.542328
1610,Metamorphosis and Other Stories,Franz Kafka,"Fiction And Literature,Spirituality And Philos...",0.597776
1561,Metamorphosis and Other Stories,Franz Kafka,"Fiction And Literature,Spirituality And Philos...",0.598379
1501,Dharmayoddha Kalki,Kevin Missal,"Fiction And Literature,Spirituality And Philos...",0.655412
1158,Jay Vudi,Bhairav Aryal,"Nepali,Nepali Literature",0.657841


In [54]:
recommend_knn("Diary of a Wimpy Kid", df, knn, combined_features_scaled)

,Title,Author,combined_genres,Distance
341,Diary Of A Wimpy Kid ; The Ugly Truth,Jeff Kinney,"Kids And Teens,Ages 9 to 12",0.387765
362,Diary of a Wimpy Kid: Wrecking Ball,Jeff Kinney,"Kids And Teens,Ages 9 to 12",0.416114
787,Diary Of A Wimpy Kid: No Brainer,Jeff Kinney,"Kids And Teens,Fiction And Literature,Manga An...",0.417212
355,Cabin Fever,Jeff Kinney,"Kids And Teens,Ages 9 to 12",0.442148
741,Diary of an Awesome Friendly Kid: Rowley Jeffe...,Jeff Kinney,"Fiction And Literature,Kids And Teens,Manga An...",0.463581


In [55]:
recommend_knn("Jay Vudi", df, knn, combined_features_scaled)

,Title,Author,combined_genres,Distance
527,Ratna's basic Nepali dictionary,"Shyam P. Wagley, Bijay Kumar Rauniyar","Learning And Reference,Dictionaries",0.542067
1099,Ghatmandu,Kumar Nagarkoti,"Nepali,Nepali Literature,Nepali Language",0.554654
196,Arresting god in kathmandu,Samrat Upadhyay,"Fiction And Literature,Short Story,Contemporary",0.575301
1146,Damini Bhir,Rajan Mukarung,"Nepali,Nepali Literature,Nepali Language",0.576937
1156,Ijajatpatra,Sarthak Karki,"Nepali,Nepali Literature",0.580010


In [56]:
recommend_knn("Big Magic", df, knn, combined_features_scaled, n_recommendations=10)

,Title,Author,combined_genres,Distance
1835,Eat Pray Love,Elizabeth Gilbert,"History Biography And Social Science,Fiction A...",0.205838
944,The Signature of All Things,Elizabeth Gilbert,"Fiction And Literature,Historical Fiction",0.451176
457,How Emotions Are Made,Lisa Feldman Barrett,"History Biography And Social Science,Learning ...",0.456232
495,The Artist's Way,Julia Cameron,"Arts And Photography,Self Improvement And Rela...",0.502150
319,Jane Eyre,Charlotte Brontë,"Fiction And Literature,History Biography And S...",0.518982
313,The Diary of a young girl,Anne Frank,"History Biography And Social Science,Fiction A...",0.623711
229,I Don't Love You Anymore: Moving On and Living...,Rithvik Singh,"Fiction And Literature,Poetry and Prose",0.624618
1427,The One Thing: The Suprisingly Simple Truth Be...,"Gary Keller, Jay Papasan","Business And Investing,Self Improvement And Re...",0.627458
1451,Everything I Know about Love,Dolly Alderton,"Self Improvement And Relationships,History Bio...",0.639673
26,Art of Mindfulness,Emma Farrarons,"Arts And Photography,Self Improvement And Rela...",0.647875
